# SIG to GMT Conversion
This notebook will process .sig files related to protein-protein interactions into the alternate file format of a .gmt file. Date processed: July 6 2017

## Import Scripts Necessary to Run this Notebook

In [1]:
%run /home/maayanlab/Desktop/Projects/KEA3/Scripts/init.ipy

## Read in .SIG files corresponding to Database
View each SIG file to ensure everything was downloaded in proper file format.

In [2]:
KEGG_sig = pd.read_table("KEGGsig.txt", header = None, sep = ' ')

In [3]:
KEGG_sig.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,SRF,NaN,NaN,NaN,NaN,ELK1,NaN,NaN,NaN,NaN,0,binding,10592173
1,SRF,NaN,NaN,NaN,NaN,ELK4,NaN,NaN,NaN,NaN,0,binding,10592173
2,MKNK2,NaN,NaN,NaN,NaN,ATF4,NaN,NaN,NaN,NaN,0,phosphorylation,10592173
3,MKNK1,NaN,NaN,NaN,NaN,ATF4,NaN,NaN,NaN,NaN,0,phosphorylation,10592173
4,RPS6KA6,NaN,NaN,NaN,NaN,ATF4,NaN,NaN,NaN,NaN,0,phosphorylation,10592173


In [4]:
#convert a SIG file to a GMT file
#Create two versions of df 'd1' --> one flipped and one which is in original order
d1 = {'protein_1': KEGG_sig[0], 'protein_2': KEGG_sig[5]}
df = pd.DataFrame(data = d1)
df.drop_duplicates(inplace = True)

d2 = {'protein_1': KEGG_sig[5], 'protein_2': KEGG_sig[0]}
df_flipped = pd.DataFrame(data = d2)
df_flipped.drop_duplicates(inplace = True)

#Make 'df' and 'df_flipped' into a single frame to account for opposite interactions
extend = pd.concat([df, df_flipped])

#Drop any duplicates and set index according to protein 1 so that we can aggregate 
#all interacting protein 2's by a shared interaction with protein one
extend.drop_duplicates(inplace = True)
extend.set_index('protein_1', inplace = True)

gmt = extend.groupby('protein_1').agg(lambda x: tuple(x))

# Create column representing counts of protein interactions per protein
gmt['interactions'] = [int(len(lst)) for protein, lst in gmt['protein_2'].iteritems()]

# Sort proteins from max to min according to number of protein iteractions
gmt.sort_values(by = ['interactions'], ascending= False, inplace=True)

#Gain indexes of gene sets with 5 or more proteins and less than 2000 protein interactions
indices = [index for index, rowData in gmt.interactions.iteritems() if rowData >= 5 and rowData < 2000]

#filter dataframe by these indices
gmt = gmt.loc[indices]

gmt_2 = gmt.copy()

#reset index and insert a column for the description
gmt_2.insert(0, 'Description', 'No Description')
gmt_2.reset_index(inplace = True)

#Drop columns not needed in GMT and join all protein interactions by a tab
gmt_2['merged'] = ['\t'.join(x) for x in gmt_2['protein_2']]
gmt_2.drop('protein_2', axis = 1, inplace = True)
gmt_2.drop('interactions', axis = 1, inplace = True)

#create a dictionary and store in it rowData corresponding to each protein
gmt_d = dict([(key, '') for key in gmt_2.index])

# loop through rows with iterrows()
for index, rowData in gmt_2.iterrows():
    line = ('\t'.join(rowData))
    gmt_d[index] = line

In [5]:
#Transfer tab-separated info into a new gmt file
with open('KEGG_ppi.gmt', 'w') as openfile:
    for index in gmt_d:
        openfile.write(str(gmt_d[index]) + '\n')

In [6]:
genes_term = gmt.interactions

len(genes_term)

879

In [7]:
avg_num_terms = genes_term.mean(axis = 0)
avg_num_terms

16.118316268486918

In [8]:
stat_df = extend.loc[indices]
len(stat_df.protein_2.unique())

1702